# Spinal chord segmentation using UNET
## This notebook has the follwing topics

1. Dataset preprocessing
2. Create dataloader¶
3. UNET architecture
4. DICE loss
5. Training the model 


In [1]:
from PIL import Image
from matplotlib import pyplot as plt
from glob import glob as glob
import numpy as np
import cv2
import random
import os
import time
import copy
from collections import defaultdict
from tqdm import tqdm as tqdm

import torch
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch.nn.functional as F
from torchvision import transforms
from torchvision import models
import torch.nn as nn
from torchsummary import summary

# 1. Dataset preprocessing 

In [2]:
## GLOBAL SETTINGS

random.seed(0)
im_w_training = 256
im_h_training = 256
b_size = 8
EPOCHS = 40
num_class = 6
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
test_train_split = .95

In [3]:
## HELPER FUNCTIONS


def make_dataset(dir):
    images = []
    assert os.path.isdir(dir), '%s is not a valid directory' % dir

    for root, _, fnames in sorted(os.walk(dir)):
        for fname in fnames:
            if is_image_file(fname):
                path = os.path.join(root, fname)
                images.append(path)

    return images


def is_image_file(filename):
    return any(filename.endswith(extension) for extension in IMG_EXTENSIONS)


IMG_EXTENSIONS = [
    '.jpg', '.JPG', '.jpeg', '.JPEG', '.png', '.PNG', '.ppm', '.PPM', '.bmp',
    '.BMP', '.tiff'
]

In [4]:
## TRAIN TEST SPLIT

org_imgs = glob(
    '../experiments/dataset/05_Final_Ground_Truth_Data/Composite_Images/*')
lable_imgs = glob(
    '../experiments/dataset/05_Final_Ground_Truth_Data/Label_Images/*')

temp = list(zip(org_imgs, lable_imgs))
random.shuffle(temp)

org_imgs, lable_imgs = zip(*temp)

split_size = int(test_train_split * len(org_imgs))

train_org_imgs, val_org_imgs = org_imgs[0:split_size], org_imgs[split_size:-1]
train_lable_imgs, val_lable_imgs = lable_imgs[0:split_size], lable_imgs[
    split_size:-1]

In [5]:
print('Total training samples --> {}'.format(len(train_org_imgs)))
print('Total validation samples --> {}'.format(len(val_org_imgs)))

Total training samples --> 1467
Total validation samples --> 77


# 2. Create dataloader

In [6]:
class RegularDataset(Dataset):
    def __init__(self, original_img_paths, original_seg_paths, augment):

        self.transforms = augment

        self.img_width = im_w_training
        self.img_height = im_h_training

        self.A_paths = original_img_paths
        self.B_paths = original_seg_paths

        self.dataset_size = len(self.A_paths)

    def __getitem__(self, index):

        # input A (RGB source)
        A_path = self.A_paths[index]
        A = Image.open(A_path)
        A = A.resize((self.img_width, self.img_height), Image.BICUBIC)
        A_tensor = self.transforms(A)

        # input B (seg map target)
        B_path = self.B_paths[index]
        B = self.parsing_embedding(B_path)  # channel(20), H, W
        B_tensor = torch.from_numpy(B)

        input_dict = {
            'source_img': A_tensor,
            'target_img': B_tensor,
            'source_img_path': A_path,
            'target_img_path': B_path
        }

        return input_dict

    def parsing_embedding(self, img_path):
        test_img = Image.open(img_path)
        test_img = test_img.resize((self.img_width, self.img_height),
                                   Image.BICUBIC)
        test_img = np.array(test_img)
        unique_channel_values = [0, 50, 100, 150, 200, 250]
        parse_emb = []

        for each_channel_val in unique_channel_values:
            parse_emb.append(
                (test_img == each_channel_val).astype(np.float32).tolist())

        parse = np.array(parse_emb).astype(np.float32)
        return parse

    def __len__(self):
        return len(self.A_paths)

    def name(self):
        return 'RegularDataset'

In [7]:
# DEFINE AUGMENTATION

augment = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.ColorJitter(hue=.05, saturation=.05),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20, resample=Image.BILINEAR),
    transforms.ToTensor(),
    transforms.Normalize((0.5, ), (0.5, ))
])

train_dataset = RegularDataset(train_org_imgs, train_lable_imgs, augment)
val_dataset = RegularDataset(val_org_imgs, val_lable_imgs, augment)

# CREATE DATALOADER
train_dataloader = DataLoader(train_dataset,
                              shuffle=True,
                              drop_last=False,
                              num_workers=6,
                              batch_size=b_size,
                              pin_memory=True)
val_dataloader = DataLoader(val_dataset,
                            shuffle=True,
                            drop_last=False,
                            num_workers=6,
                            batch_size=b_size,
                            pin_memory=True)

# FOR DEBUGGING
print("Checking  the dimension and type of data")
for key in train_dataset[0].keys():
    try:
        x = train_dataset[0][key]
        print("name of the input and shape -- > ", key, x.shape)
        print("type,dtype,and min max -- >", type(x), x.dtype, torch.min(x),
              torch.max(x))
    except Exception as e:
        print("name of the input -- > ", key, train_dataset[0][key])
    print('----------------')

Checking  the dimension and type of data
name of the input and shape -- >  source_img torch.Size([1, 256, 256])
type,dtype,and min max -- > <class 'torch.Tensor'> torch.float32 tensor(-1.) tensor(0.6471)
----------------
name of the input and shape -- >  target_img torch.Size([6, 256, 256])
type,dtype,and min max -- > <class 'torch.Tensor'> torch.float32 tensor(0.) tensor(1.)
----------------
name of the input -- >  source_img_path ../experiments/dataset/05_Final_Ground_Truth_Data/Composite_Images/C1_0433_D5.png
----------------
name of the input -- >  target_img_path ../experiments/dataset/05_Final_Ground_Truth_Data/Label_Images/L1_0359_D4.png
----------------


# 3. UNET architecture

In [8]:


def convrelu(in_channels, out_channels, kernel, padding):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel, padding=padding),
        nn.ReLU(inplace=True),
    )


base_model = models.resnet18(pretrained=True)
base_model.conv1 = torch.nn.Conv2d(1,
                                   64,
                                   kernel_size=(7, 7),
                                   stride=(2, 2),
                                   padding=(3, 3),
                                   bias=False)


class ResNetUNet(nn.Module):
    def __init__(self, n_class):
        super().__init__()

#         self.base_model = models.resnet18(pretrained=False)
#         self.base_model.conv1 = torch.nn.Conv2d(1,
#                                                 64,
#                                                 kernel_size=(7, 7),
#                                                 stride=(2, 2),
#                                                 padding=(3, 3),
#                                                 bias=False)

        self.base_layers = list(base_model.children())

        self.layer0 = nn.Sequential(
            *self.base_layers[:3])  # size=(N, 64, x.H/2, x.W/2)
        self.layer0_1x1 = convrelu(64, 64, 1, 0)
        self.layer1 = nn.Sequential(
            *self.base_layers[3:5])  # size=(N, 64, x.H/4, x.W/4)
        self.layer1_1x1 = convrelu(64, 64, 1, 0)
        self.layer2 = self.base_layers[5]  # size=(N, 128, x.H/8, x.W/8)
        self.layer2_1x1 = convrelu(128, 128, 1, 0)
        self.layer3 = self.base_layers[6]  # size=(N, 256, x.H/16, x.W/16)
        self.layer3_1x1 = convrelu(256, 256, 1, 0)
        self.layer4 = self.base_layers[7]  # size=(N, 512, x.H/32, x.W/32)
        self.layer4_1x1 = convrelu(512, 512, 1, 0)

        self.upsample = nn.Upsample(scale_factor=2,
                                    mode='bilinear',
                                    align_corners=True)

        self.conv_up3 = convrelu(256 + 512, 512, 3, 1)
        self.conv_up2 = convrelu(128 + 512, 256, 3, 1)
        self.conv_up1 = convrelu(64 + 256, 256, 3, 1)
        self.conv_up0 = convrelu(64 + 256, 128, 3, 1)

        self.conv_original_size0 = convrelu(1, 64, 3, 1)
        self.conv_original_size1 = convrelu(64, 64, 3, 1)
        self.conv_original_size2 = convrelu(64 + 128, 64, 3, 1)

        self.conv_last = nn.Conv2d(64, n_class, 1)

    def forward(self, input):
        x_original = self.conv_original_size0(input)
        x_original = self.conv_original_size1(x_original)

        layer0 = self.layer0(input)
        layer1 = self.layer1(layer0)
        layer2 = self.layer2(layer1)
        layer3 = self.layer3(layer2)
        layer4 = self.layer4(layer3)

        layer4 = self.layer4_1x1(layer4)
        x = self.upsample(layer4)
        layer3 = self.layer3_1x1(layer3)
        x = torch.cat([x, layer3], dim=1)
        x = self.conv_up3(x)

        x = self.upsample(x)
        layer2 = self.layer2_1x1(layer2)
        x = torch.cat([x, layer2], dim=1)
        x = self.conv_up2(x)

        x = self.upsample(x)
        layer1 = self.layer1_1x1(layer1)
        x = torch.cat([x, layer1], dim=1)
        x = self.conv_up1(x)

        x = self.upsample(x)
        layer0 = self.layer0_1x1(layer0)
        x = torch.cat([x, layer0], dim=1)
        x = self.conv_up0(x)

        x = self.upsample(x)
        x = torch.cat([x, x_original], dim=1)
        x = self.conv_original_size2(x)

        out = self.conv_last(x)

        return out

In [9]:
model = ResNetUNet(num_class)
model = model.to(device)

summary(model, input_size=(1, im_w_training, im_h_training))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 256, 256]             640
              ReLU-2         [-1, 64, 256, 256]               0
            Conv2d-3         [-1, 64, 256, 256]          36,928
              ReLU-4         [-1, 64, 256, 256]               0
            Conv2d-5         [-1, 64, 128, 128]           3,136
       BatchNorm2d-6         [-1, 64, 128, 128]             128
              ReLU-7         [-1, 64, 128, 128]               0
         MaxPool2d-8           [-1, 64, 64, 64]               0
            Conv2d-9           [-1, 64, 64, 64]          36,864
      BatchNorm2d-10           [-1, 64, 64, 64]             128
             ReLU-11           [-1, 64, 64, 64]               0
           Conv2d-12           [-1, 64, 64, 64]          36,864
      BatchNorm2d-13           [-1, 64, 64, 64]             128
             ReLU-14           [-1, 64,

# 4. DICE loss

In [10]:
def dice_loss(pred, target, smooth=1.):
    pred = pred.contiguous()
    target = target.contiguous()

    intersection = (pred * target).sum(2).sum(2)

    loss = (1 - ((2. * intersection + smooth) /
                 (pred.sum(2).sum(2) + target.sum(2).sum(2) + smooth)))

    return loss.mean()


def calc_loss(pred, target, metrics, bce_weight=0.5):
    bce = F.binary_cross_entropy_with_logits(pred, target)

    pred = F.sigmoid(pred)
    dice = dice_loss(pred, target)

    loss = bce * bce_weight + dice * (1 - bce_weight)

    metrics['bce'] += bce.data.cpu().numpy() * target.size(0)
    metrics['dice'] += dice.data.cpu().numpy() * target.size(0)
    metrics['loss'] += loss.data.cpu().numpy() * target.size(0)

    return loss


def print_metrics(metrics, epoch_samples, phase):
    outputs = []
    for k in metrics.keys():
        outputs.append("{}: {:4f}".format(k, metrics[k] / epoch_samples))

    print("{}: {}".format(phase, ", ".join(outputs)))

# 5. Train the model

In [11]:
def train_model(model, optimizer, scheduler, num_epochs=25):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = 1e10

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        since = time.time()

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                for param_group in optimizer.param_groups:
                    print("LR", param_group['lr'])

                model.train()  # Set model to training mode
                dataloader = train_dataloader
                dataloader_length = len(train_dataset) // b_size + 1
            else:
                model.eval()  # Set model to evaluate mode
                dataloader = val_dataloader
                dataloader_length = len(val_dataset) // b_size + 1
            metrics = defaultdict(float)
            epoch_samples = 0

            for i, results in tqdm(enumerate(dataloader),
                                   total=dataloader_length):
                source_img = results['source_img'].float().cuda()
                target_parse = results['target_img'].float().cuda()

                inputs = source_img
                labels = target_parse

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = calc_loss(outputs, labels, metrics)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                epoch_samples += inputs.size(0)

            print_metrics(metrics, epoch_samples, phase)
            epoch_loss = metrics['loss'] / epoch_samples

            # deep copy the model
            if phase == 'val' and epoch_loss < best_loss:

                best_loss = epoch_loss
                best_model_wts = copy.deepcopy(model.state_dict())

        time_elapsed = time.time() - since
        print('total time taken to run one epoch -- {:.0f}m {:.0f}s'.format(
            time_elapsed // 60, time_elapsed % 60))

        print(
            '---------------------------------------------------------------------------------'
        )

    print('Best val loss: {:4f}'.format(best_loss))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [13]:
optimizer_ft = optim.Adam(filter(lambda p: p.requires_grad,
                                 model.parameters()),
                          lr=0.0002,
                          betas=[0.5, 0.999])

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=30, gamma=0.1)
model = train_model(model, optimizer_ft, exp_lr_scheduler, num_epochs=40)

Epoch 0/39
----------
LR 0.0002


/home/deep/.virtualenvs/pytorch/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:118: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
  0%|          | 0/184 [00:00<?, ?it/s]/home/deep/.virtualenvs/pytorch/lib/python3.8/site-packages/torch/nn/functional.py:1625: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
100%|██████████| 184/184 [00:47<00:00,  3.91it/s]

train: bce: 0.064168, dice: 0.579442, loss: 0.321805



100%|██████████| 10/10 [00:01<00:00,  5.13it/s]

val: bce: 0.042415, dice: 0.500260, loss: 0.271337
total time taken to run one epoch -- 0m 49s
---------------------------------------------------------------------------------
Epoch 1/39
----------
LR 0.0002


...

val: bce: 0.049409, dice: 0.503025, loss: 0.276217
total time taken to run one epoch -- 0m 47s
---------------------------------------------------------------------------------
Epoch 39/39
----------
LR 2e-05



100%|██████████| 184/184 [00:46<00:00,  3.94it/s]

train: bce: 0.033189, dice: 0.408124, loss: 0.220657



100%|██████████| 10/10 [00:01<00:00,  6.62it/s]

val: bce: 0.049381, dice: 0.498474, loss: 0.273928
total time taken to run one epoch -- 0m 48s
---------------------------------------------------------------------------------
Best val loss: 0.261659


In [16]:
torch.save(model.state_dict(), './model_final_dict.pt')

In [17]:
torch.save(model,'model_final.pt')